# 处理缺失值

现实中使用的数据很少是干净的，数据缺失现象普遍存在。本节将介绍一些处理缺失值的通用方法。这些缺失值在数据集中的存在形式主要有两种：  
**None**、**NaN**。

## 1. 'None': Python 内置对象类型的缺失值
Pandas 可以使用的第一种缺失值标签是 None，它是一个 Python 内置对象 NoneType 的唯一值，经常在代码中表示缺失值。None 只能用于 objects 类型。

In [1]:
import numpy as np
import pandas as pd

In [2]:
vals1 = np.array([1, None, 3, 4])
vals1

array([1, None, 3, 4], dtype=object)

这里 dtype=object 表示 NumPy 认为由于这个数组是 Python 对象构成的，因此将其类型判断为 object。进行快速操作时，object类型比其他类型数组要耗费更多的资源。

In [3]:
for dtype in ['object', 'int']:
    print("dtype =", dtype)
    %timeit np.arange(1E6, dtype=dtype).sum()
    print()

dtype = object
33.3 ms ± 948 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)

dtype = int
1.25 ms ± 2.85 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)



同时，由于 Python 没有定义 object 的常用统计方法，因此在调用一些统计方法时会报错：

In [9]:
vals1.sum()

TypeError: unsupported operand type(s) for +: 'int' and 'NoneType'

## 2. ``NaN``: 数值类型的缺失值  
另一种缺失值的标签是 NaN(nan)，全称为 Not a Number，不是一个数字。在 NumPy 和 Pandas 写作 nan 或者 NaN。

In [5]:
vals2 = np.array([1, np.nan, 3, 4]) 
vals2

array([ 1., nan,  3.,  4.])

nan是一个特殊的浮点数。

In [6]:
vals2.dtype

dtype('float64')

nan 就像一个病毒，任何和它接触过的数据（一起运算）都会被同化

In [7]:
1 + np.nan

nan

In [8]:
0 *  np.nan

nan

In [9]:
vals2.sum(), vals2.min(), vals2.max()

(nan, nan, nan)

NumPy 提供了特殊的方法，可以忽略nan的存在

In [10]:
np.nansum(vals2), np.nanmin(vals2), np.nanmax(vals2)

(8.0, 1.0, 4.0)

## 3. 处理缺失值

Pandas 基本上把 None 和 NaN 看成可以等价的缺失值形式。Pandas 提供了一些方法来检测、删除、替换数据集中的缺失值。包括：  

- ``isnull()`` ：判断 Series、DataFrame 里的每一个数据是不是 NaN 或者 None （NaN 或者 None 返回 True）
- ``notnull()``：判断 Series、DataFrame 里的每一个数据是不是 NaN 或者 None （NaN 或者 None 返回 False）
- ``dropna()``：返回一个删除缺失值的数据集
- ``fillna()``：返回一个填充了缺失值的数据副本

### 3.1 发现缺失值

In [9]:
data = pd.Series([1, np.nan, 'hello', None])
data

0        1
1      NaN
2    hello
3     None
dtype: object

In [11]:
print(data.notnull())

0     True
1    False
2     True
3    False
dtype: bool


In [15]:
data[data.notnull()]

0        1
2    hello
dtype: object

### 4.2 删除缺失值

In [30]:
data.dropna()

0        1
2    hello
dtype: object

In [12]:
df = pd.DataFrame([[1,      np.nan, 2],
                   [2,      3,      5],
                   [np.nan, 4,      6]])
df

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


我们无法从 DataFrame 中单独删除一个值，要么删除缺失值所在的整行，要么删除缺失值所在的整列。默认情况下，dropna() 会删除整行。

In [13]:
df.dropna()

,0,1,2
1,2.0,3.0,5


In [14]:
df

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


In [18]:
df.dropna(axis=1) # 通过修改axis参数，删除整列

,2
0,2
1,5
2,6


有时候我们只希望删除全部是缺失值的行或者列，或者绝大多数是缺失值的行或者列。我们可以采用参数 how 、thresh来满足。  

In [17]:
df[3] = np.nan
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [20]:
df.dropna(axis=1, how='all') # how = 'all' 删除全部是缺失值的行或者列 how = 'any' 删除存在缺失值的行或者列

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


In [21]:
df.dropna(axis=0, thresh=3) # thresh 可以设置行或列中非缺失值的最小数量。

,0,1,2,3
1,2.0,3.0,5,NaN


### 4.3 填充缺失值

使用有效数值替换缺失值

In [22]:
data = pd.Series([1, np.nan, 2, None, 3], index=list('abcde'))
data

a    1.0
b    NaN
c    2.0
d    NaN
e    3.0
dtype: float64

In [18]:
data.fillna(0) # 使用0来替换缺失值

0        1
1        0
2    hello
3        0
dtype: object

In [24]:
# forward-fill
data.fillna(method='ffill') # 使用缺失值前面的有效数从前往后填充

a    1.0
b    1.0
c    2.0
d    2.0
e    3.0
dtype: float64

In [26]:
# back-fill  ffill
data.fillna(method='bfill') # 使用缺失值后面的有效数从后往前填充

a    1.0
b    2.0
c    2.0
d    3.0
e    3.0
dtype: float64

DataFrame 的操作类似，只是需要设置 axis 参数

In [19]:
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [22]:
df.fillna(method='ffill', axis=1)

,0,1,2,3
0,1.0,1.0,2.0,2.0
1,2.0,3.0,5.0,5.0
2,NaN,4.0,6.0,6.0
